In [ ]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import pyperclip

import re
import time
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format
import seaborn as sns

import koreanize_matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from wordcloud import WordCloud

font_path = '/Users/jieunlee/Documents/3_TECH!T_202305/TECH!T_dataton/nanum-barun-gothic/NanumBarunGothic.ttf'  # 각자 경로에 맞게
fontprop = fm.FontProperties(fname=font_path, size=18)

# **Crawling_NaverWebtoon(한국)**

### **직접 수집한 한국url에서 추가 크롤링 진행**

In [ ]:
data = pd.read_csv('/Users/jieunlee/Documents/3_TECH!T_202305/TECH!T_dataton/crawling_total_한국url_908_230913.csv')

In [ ]:
# 'Url' 및 '국가' 데이터 잘 수집되었는지 확인
data[(data['작가'] != 'WEBTOON') & ((data['작가'] != 'Various Artists')) & (data['Url'].isna() & (data['국가'].isna()))]

,Unnamed: 0,제목,작가,장르,좋아요수,조회수,구독수,별점,회차,Url,국가,기타url


In [ ]:
add = pd.DataFrame(columns=['한국제목', '한국작가명', '회차(한국)', '별점(한국)', '관심수(한국)', '장르(한국)', '연재시작일', '마지막연재일', '완결여부'])

In [ ]:
user_id = ''
user_pw = ''

for i, Url in enumerate(data['Url']):
    
    # 네이버웹툰
    if str(Url).startswith('https://comic.naver.com/'):

        url = Url
        driver = webdriver.Chrome()
        driver.get(url)

        try:
            # login
            driver.find_element(By.XPATH, '//*[@id="id"]').click()
            pyperclip.copy(user_id)
            ActionChains(driver).key_down(Keys.COMMAND).send_keys('v').key_up(Keys.COMMAND).perform()
            # driver.find_element(By.XPATH, '//*[@id="id"]').send_keys(Keys.COMMAND, 'v')
            time.sleep(0.5)

            driver.find_element(By.XPATH, '//*[@id="pw"]').click()
            pyperclip.copy(user_pw)
            ActionChains(driver).key_down(Keys.COMMAND).send_keys('v').key_up(Keys.COMMAND).perform()
            # driver.find_element(By.XPATH, '//*[@id="id"]').send_keys(Keys.COMMAND, 'v')
            time.sleep(0.5)
            
            # 보안을 위한 단계
            secure='blank'
            pyperclip.copy(secure)

            driver.find_element(By.XPATH, '//*[@id="log.login"]').click()
            
            time.sleep(1)
        
        except:
            pass

        time.sleep(0.5)

        title = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[1]/div/h2""").text.replace('\n휴재', '')
        author = ' '.join(driver.find_element(By.XPATH, f"""//*[@id="content"]/div[1]/div/div[1]""").text.replace('∙\n', '').split('\n')[:-2])
        favorite = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[2]/div/button[1]/span[2]""").text.replace(',', '')
        genre = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[1]/div/div[2]/div/div""").text.replace('#', '').replace('\n', ',')

        try: 
            episode = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/div[1]/div[1]""").text.replace('총', '').replace('화', '').strip()
            
            # start_date
            driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/div[1]/div[2]/button[2]""").click()
            time.sleep(0.5)
            start_date = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/ul/li[1]/a/div[2]/div/span[2]""").text
            star_start = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/ul/li[1]/a/div[2]/div/span[1]/span""").text
            # end_date
            driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/div[1]/div[2]/button[1]""").click()
            time.sleep(0.5)
            end_date = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/ul/li[1]/a/div[2]/div/span[2]""").text
            star_end = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/ul/li[1]/a/div[2]/div/span[1]/span""").text

            # 별점
            star_score = round((float(star_start) + float(star_end)) / 2, 2)

        except:
            episode = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/div[2]/div[1]""").text.replace('총', '').replace('화', '').strip()
        
            # start_date
            driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/div[2]/div[2]/button[2]""").click()
            time.sleep(0.5)
            start_date = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/ul/li[1]/a/div[2]/div/span[2]""").text
            star_start = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/ul/li[1]/a/div[2]/div/span[1]/span""").text
            # end_date
            driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/div[2]/div[2]/button[1]""").click()
            time.sleep(0.5)
            end_date = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/ul/li[1]/a/div[2]/div/span[2]""").text
            star_end = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[3]/ul/li[1]/a/div[2]/div/span[1]/span""").text

            # 별점
            star_score = round((float(star_start) + float(star_end)) / 2, 2)

        # 완결여부
        finished_info = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[1]/div/div[1]""").text
        finished = 'True' if '완결' in finished_info else 'False'

        add.loc[len(add)] = [title, author, episode, star_score, favorite, genre, start_date, end_date, finished]

        time.sleep(0.5)


    # 네이버시리즈
    elif str(Url).startswith('https://series.naver.com/'):

        url = Url
        driver = webdriver.Chrome()
        driver.get(url)

        try:
            # login
            driver.find_element(By.XPATH, '//*[@id="id"]').click()
            pyperclip.copy(user_id)
            ActionChains(driver).key_down(Keys.COMMAND).send_keys('v').key_up(Keys.COMMAND).perform()
            # driver.find_element(By.XPATH, '//*[@id="id"]').send_keys(Keys.COMMAND, 'v')
            time.sleep(0.5)

            driver.find_element(By.XPATH, '//*[@id="pw"]').click()
            pyperclip.copy(user_pw)
            ActionChains(driver).key_down(Keys.COMMAND).send_keys('v').key_up(Keys.COMMAND).perform()
            # driver.find_element(By.XPATH, '//*[@id="id"]').send_keys(Keys.COMMAND, 'v')
            time.sleep(0.5)
            
            # 보안을 위한 단계
            secure='blank'
            pyperclip.copy(secure)

            driver.find_element(By.XPATH, '//*[@id="log.login"]').click()

            time.sleep(1)
        
        except:
            pass

        time.sleep(0.5)
        
        try: 
            title = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[1]/h2""").text
            favorite = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[1]/div[2]/ul/li[2]/div/a/em""").text.replace(',', '')
            star_score = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[1]/div[1]/em""").text
        
        except:
            title = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[2]/h2""").text
            favorite = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[2]/div[2]/ul/li[2]/div/a/em""").text.replace(',', '')
            star_score = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[2]/div[1]/em""").text
        
        # author
        author_write = driver.find_element(By.XPATH, f"""//*[@id="content"]/ul[1]/li/ul/li[3]""").text.replace('글', '글 ')
        author_draw = driver.find_element(By.XPATH, f"""//*[@id="content"]/ul[1]/li/ul/li[4]""").text.replace('그림', '그림 ')
        author = author_write + ' ' + author_draw
        
        episode = driver.find_element(By.XPATH, f"""//*[@id="content"]/h5/strong""").text
        genre = driver.find_element(By.XPATH, f"""//*[@id="content"]/ul[1]/li/ul/li[2]/span/a""").text

        # start_date
        driver.find_element(By.XPATH, f"""//*[@id="content"]/table/thead/tr/th[2]/div/button""").click()
        time.sleep(0.5)
        start_date = driver.find_element(By.XPATH, f"""//*[@id="volumeList"]/tr[1]/td[1]/div/em""").text.replace('(', '').replace('.)', '')
        # end_date
        driver.find_element(By.XPATH, f"""//*[@id="content"]/table/thead/tr/th[1]/div/button""").click()
        time.sleep(0.5)
        end_date = driver.find_element(By.XPATH, f"""//*[@id="volumeList"]/tr[1]/td[1]/div/em""").text.replace('(', '').replace('.)', '')
        
        # 완결여부
        finished_info = driver.find_element(By.XPATH, f"""//*[@id="content"]/ul[1]/li/ul/li[1]/span""").text
        finished = 'True' if finished_info == '완결' else 'False'

        add.loc[len(add)] = [title, author, episode, star_score, favorite, genre, start_date, end_date, finished]

        time.sleep(0.5)
    
    else:
        title, author, episode, star_score, favorite, genre, start_date, end_date, finished = np.nan, np.nan , np.nan , np.nan, np.nan , np.nan , np.nan , np.nan, np.nan 
        add.loc[len(add)] = [title, author, episode, star_score, favorite, genre, start_date, end_date, finished]

    print(i)

823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908


In [ ]:
add.to_csv('add_908.csv', index=False)

In [ ]:
len(add)

908

In [ ]:
len(data)

908

In [ ]:
add.tail()

,한국제목,한국작가명,회차(한국),별점(한국),관심수(한국),장르(한국),연재시작일,마지막연재일,완결여부
903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
905,패밀리맨,글/그림 정필원,46,9.91,17600,"드라마,완결드라마",09.07.31,10.07.22,True
906,스마일브러시,글/그림 와루,39,9.94,13609,"일상,완결무료,완결일상",10.03.31,10.06.26,True
907,골방환상곡,글 워니 그림 심윤수,351,9.88,12708,"일상,완결무료,완결일상",05.12.12,08.12.24,True


In [ ]:
data.tail()

,Unnamed: 0,제목,작가,장르,좋아요수,조회수,구독수,별점,회차,Url,국가,기타url
903,534,ฮาเร็ม ซะเมื่อไหร่กันเล่า!,ศุภชัย จิรคุปต์,ตลก,"86,001",5.4M,"66,249",9.43,27,NaN,태국,NaN
904,535,ตำนานทางรถไฟหลังโรงเรียน,คงไคย,ระทึกขวัญ,"76,032",3.4M,"81,250",9.69,1,NaN,태국,NaN
905,536,ซุปเปอร์คุณพ่อ,พิลวอน จอง,ดราม่า,"51,789",2.8M,"33,045",9.66,42,https://comic.naver.com/webtoon/list?titleId=7...,NaN,NaN
906,537,ฝีแปรงแย้มยิ้ม,วารู,อบอุ่นหัวใจ,"78,819",4.3M,"48,520",9.51,36,https://comic.naver.com/webtoon/list?titleId=1...,NaN,NaN
907,538,ธรรมดาไดอารี่,วอนี,ตลก,"52,515",2.9M,"30,741",9.00,38,https://comic.naver.com/webtoon/list?titleId=1...,NaN,NaN


### **concat**

In [ ]:
data_concat = pd.concat([data, add], axis=1)

In [ ]:
data_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  908 non-null    int64  
 1   제목          908 non-null    object 
 2   작가          908 non-null    object 
 3   장르          908 non-null    object 
 4   좋아요수        908 non-null    object 
 5   조회수         908 non-null    object 
 6   구독수         908 non-null    object 
 7   별점          908 non-null    float64
 8   회차          908 non-null    int64  
 9   Url         598 non-null    object 
 10  국가          299 non-null    object 
 11  기타url       32 non-null     object 
 12  한국제목        598 non-null    object 
 13  한국작가명       598 non-null    object 
 14  회차(한국)      598 non-null    object 
 15  별점(한국)      598 non-null    object 
 16  관심수(한국)     598 non-null    object 
 17  장르(한국)      598 non-null    object 
 18  연재시작일       598 non-null    object 
 19  마지막연재일      598 non-null    o

In [ ]:
data_concat.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
data_concat.to_csv('crawling_total_한국정보추가_908_230913.csv', index=False)

### **Crawling_작가의 전작 개수**

In [ ]:
data = pd.read_csv('crawling_total_한국정보추가_908_230913.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  908 non-null    int64  
 1   제목          908 non-null    object 
 2   작가          908 non-null    object 
 3   장르          908 non-null    object 
 4   좋아요수        908 non-null    object 
 5   조회수         908 non-null    object 
 6   구독수         908 non-null    object 
 7   별점          908 non-null    float64
 8   회차          908 non-null    int64  
 9   Url         598 non-null    object 
 10  국가          299 non-null    object 
 11  기타url       32 non-null     object 
 12  한국제목        598 non-null    object 
 13  한국작가명       598 non-null    object 
 14  회차(한국)      598 non-null    float64
 15  별점(한국)      598 non-null    float64
 16  관심수(한국)     598 non-null    object 
 17  장르(한국)      598 non-null    object 
 18  연재시작일       598 non-null    object 
 19  마지막연재일      598 non-null    o

In [ ]:
cnt = pd.DataFrame(columns=['작가전작개수'])

In [ ]:
user_id = ''
user_pw = ''

for i, Url in enumerate(data['Url'][701:]):
    
    # 네이버웹툰
    if str(Url).startswith('https://comic.naver.com/'):

        url = Url
        driver = webdriver.Chrome()
        driver.get(url)
        
        # login
        try:
            driver.find_element(By.XPATH, '//*[@id="id"]').click()
            pyperclip.copy(user_id)
            ActionChains(driver).key_down(Keys.COMMAND).send_keys('v').key_up(Keys.COMMAND).perform()
            # driver.find_element(By.XPATH, '//*[@id="id"]').send_keys(Keys.COMMAND, 'v')
            time.sleep(0.5)

            driver.find_element(By.XPATH, '//*[@id="pw"]').click()
            pyperclip.copy(user_pw)
            ActionChains(driver).key_down(Keys.COMMAND).send_keys('v').key_up(Keys.COMMAND).perform()
            # driver.find_element(By.XPATH, '//*[@id="id"]').send_keys(Keys.COMMAND, 'v')
            time.sleep(0.5)
            
            # 보안을 위한 단계
            secure='blank'
            pyperclip.copy(secure)

            driver.find_element(By.XPATH, '//*[@id="log.login"]').click()
            
            time.sleep(1)
        
        except:
            pass

        time.sleep(0.5)

        # 네이버웹툰 전작 개수 크롤링
        container_1 = driver.find_element(By.XPATH, """//*[@id="container"]/div/div[2]/div[3]""")
        container_2 = driver.find_element(By.XPATH, """//*[@id="container"]/div/div[2]/div[4]""")

        # 작가의 다른 작품
        if container_1.text.split('\n')[0] == '작가의 다른 작품':
            
            try:
                # 전작 개수가 3개 이상
                pages = int(container_1.text.split('\n')[-1][-1])

                for page in range(pages - 1):
                    click = driver.find_element(By.XPATH, """//*[@id="container"]/div/div[2]/div[3]/a""")
                    click.click()
                    time.sleep(0.5)
                
                try:
                    other_products = int(container_1.text.split('\n')[-5])
                except:
                    other_products = int(container_1.text.split('\n')[-6])
                    
            except:
                # 전작 개수가 3개 이하
                try:
                    other_products = int(container_1.text.split('\n')[-3])
                except:
                    other_products = int(container_1.text.split('\n')[-4])
                    
            total_products = other_products + 1
            total_products

        # 관련 상품 -> 작가의 다른 작품
        elif container_1.text.split('\n')[0] != '작가의 다른 작품': 

            if container_2.text.split('\n')[0] == '작가의 다른 작품':

                try:
                
                    # 전작 개수가 3개 이상
                    pages = int(container_2.text.split('\n')[-1][-1])

                    # 마지막 페이지까지 클릭
                    for page in range(pages - 1):
                        click = driver.find_element(By.XPATH, """//*[@id="container"]/div/div[2]/div[4]/a""")
                        click.click()
                        time.sleep(0.5)
                    try:
                        other_products = int(container_2.text.split('\n')[-5])
                    except:
                        other_products = int(container_2.text.split('\n')[-6])
                        
                except:
                    # 전작 개수가 3개 이하
                    try:
                        other_products = int(container_2.text.split('\n')[-3])
                    except:
                        other_products = int(container_2.text.split('\n')[-4])

                total_products = other_products + 1
                total_products
            
            # 전작 개수가 0개
            else:
                total_products = 1


    # 네이버시리즈
    elif str(Url).startswith('https://series.naver.com/'):

        url = Url
        driver = webdriver.Chrome()
        driver.get(url)

        # login
        try:
            driver.find_element(By.XPATH, '//*[@id="id"]').click()
            pyperclip.copy(user_id)
            ActionChains(driver).key_down(Keys.COMMAND).send_keys('v').key_up(Keys.COMMAND).perform()
            # driver.find_element(By.XPATH, '//*[@id="id"]').send_keys(Keys.COMMAND, 'v')
            time.sleep(0.5)

            driver.find_element(By.XPATH, '//*[@id="pw"]').click()
            pyperclip.copy(user_pw)
            ActionChains(driver).key_down(Keys.COMMAND).send_keys('v').key_up(Keys.COMMAND).perform()
            # driver.find_element(By.XPATH, '//*[@id="id"]').send_keys(Keys.COMMAND, 'v')
            time.sleep(0.5)
            
            # 보안을 위한 단계
            secure='blank'
            pyperclip.copy(secure)

            driver.find_element(By.XPATH, '//*[@id="log.login"]').click()

            time.sleep(1)
        
        except:
            pass

        time.sleep(0.5)

        # 네이버시리즈 전작 개수 크롤링
        try:
            author = driver.find_element(By.XPATH, """//*[@id="_otherProductByPerson"]/strong[2]""").text

            # 전작 개수가 5개 이상
            try: 
                pages = driver.find_element(By.XPATH, """//*[@id="content"]/div[10]/div[2]/span""").text
                pages_list = pages.split(' ')

                other_product_1 = int(pages_list[-2]) * 5

                # 페이지 수가 2개 이하인 경우
                if int(pages_list[-1]) == 2:
                    # 마지막 페이지로 이동
                    last_page = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[10]/div[2]/span/a""")
                    last_page.click()
                    time.sleep(0.5)

                # 페이지 수가 2개 이상인 경우
                else:
                    # 마지막 페이지로 이동
                    last_page = driver.find_element(By.XPATH, f"""//*[@id="content"]/div[10]/div[2]/span/a[{int(pages_list[-2])}]""")
                    last_page.click()
                    time.sleep(0.5)

                # 마지막 페이지의 작품 개수
                other_product_2 = driver.find_element(By.XPATH, """//*[@id="content"]/div[10]""").text.split('\n')[:-1]
                other_product_2 = len(other_product_2)
                    
                total_products = other_product_1 + other_product_2 + 1

            # 전작 개수가 5개 이하
            except:
                other_product = driver.find_element(By.XPATH, """//*[@id="content"]/div[10]""")
                other_product = len(other_product.text.split('\n'))

                total_products = other_product + 1

        # 전작 개수가 0개
        except:
            author = None
            total_products = 1

    else:
        total_products = np.nan

    cnt.loc[len(cnt)] = [total_products]

    time.sleep(0.5)

    print(701+i+1, total_products)

702 2
703 1
704 3
705 nan
706 nan
707 nan
708 1
709 2
710 1
711 1
712 nan
713 3
714 nan
715 2
716 1
717 nan
718 nan
719 nan
720 nan
721 nan
722 nan
723 nan
724 nan
725 1
726 nan
727 nan
728 3
729 nan
730 nan
731 6
732 nan
733 nan
734 nan
735 2
736 6
737 2
738 nan
739 28
740 nan
741 nan
742 1
743 nan
744 nan
745 nan
746 14
747 1
748 nan
749 2
750 4
751 nan
752 nan
753 nan
754 3
755 nan
756 nan
757 nan
758 4
759 nan
760 1
761 3
762 14
763 4
764 nan
765 3
766 1
767 5
768 nan
769 nan
770 3
771 1
772 5
773 nan
774 3
775 nan
776 2
777 1
778 nan
779 nan
780 nan
781 4
782 4
783 9
784 nan
785 20
786 nan
787 1
788 nan
789 3
790 6
791 nan
792 nan
793 nan
794 7
795 nan
796 nan
797 nan
798 1
799 5
800 6
801 nan
802 3
803 5
804 14
805 nan
806 nan
807 nan
808 nan
809 11
810 nan
811 nan
812 nan
813 1
814 nan
815 1
816 nan
817 4
818 1
819 9
820 3
821 4
822 1
823 1
824 nan
825 nan
826 nan
827 nan
828 nan
829 nan
830 2
831 nan
832 nan
833 13
834 nan
835 1
836 nan
837 nan
838 nan
839 1
840 nan
841 nan
842

In [ ]:
len(cnt)

908

In [ ]:
save = cnt.to_csv('save_908.csv')

In [ ]:
data['작가전작개수'] = cnt

### **OSMU 여부**

In [ ]:
data.loc[data['장르(한국)'].notna() & data['장르(한국)'].str.contains('원작웹툰'), 'OSMU여부'] = True
data['OSMU여부'].fillna(False, inplace=True)

In [ ]:
data['OSMU여부'].value_counts()

OSMU여부
False    876
True      32
Name: count, dtype: int64

In [ ]:
data.to_csv('crawling_total_한국정보추가_908_230917.csv', index=False)